# Projeto 3 -  SCC0270 - Neural Networks and Deep Learning


### Nome: William Luis Alves Ferreira

### NºUSP: 9847599

In [1]:
from tensorflow.keras.utils import to_categorical
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Flatten
import numpy as np
import pandas as pd

#### Leitura dos dados para análise

## Questão 1

### Importando modelo

In [2]:
from tensorflow.keras.applications import MobileNetV2

Com as especificações do enunciado temos o modelo pré-treinado MobileNetV2

In [3]:
modeloMN = MobileNetV2(
    include_top = False,
    input_shape = (128,128,3),
    weights='imagenet'
)

In [4]:
for layer in modeloMN.layers:
    layer.trainable = False

In [5]:
modelo= modeloMN.output
modelo

<KerasTensor: shape=(None, 4, 4, 1280) dtype=float32 (created by layer 'out_relu')>

In [6]:
modelo= Flatten()(modelo)

In [7]:
modelo= Dense(7, activation='softmax')(modelo)

In [8]:
new_modelo= Model(
    inputs = modeloMN.input,
    outputs= modelo
)

new_modelo.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
__________________________________________________________________________________________________
Conv1_relu (ReLU)               (None, 64, 64, 32)   0           bn_Conv1[0][0]                   
______________________________________________________________________________________________

#### Processo de otimização para métrica

In [9]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy

new_modelo.compile(
    optimizer="Adam", 
    loss='categorical_crossentropy', 
    metrics=["accuracy"]
)

## Questão 2

#### Importando modelo e preprocessamento de imagens kera

In [10]:
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import mobilenet_v2

#### path dataset

In [11]:
datadir_train =  './dataset/train/' 
datadir_test =  './dataset/test/' 

#### Conjuntos de treino e teste gerados

#### Destorção imagens conjunto de treino

In [12]:
dataGen = ImageDataGenerator(
    horizontal_flip=True,
    zoom_range=0.2,
    shear_range=10,
    height_shift_range=0.1,
    width_shift_range=0.1,
    preprocessing_function=mobilenet_v2.preprocess_input
)


In [13]:
datagen_value = ImageDataGenerator(
    preprocessing_function=mobilenet_v2.preprocess_input
)

In [14]:
trainGen = dataGen.flow_from_directory(
    datadir_train,
    target_size = (128,128),
    batch_size= 32,
    class_mode = 'categorical'
)


Found 2284 images belonging to 7 classes.


In [15]:
testGen = datagen_value.flow_from_directory(
    datadir_test,
    target_size = (128,128),
    batch_size= 32,
    class_mode = 'categorical'
)

Found 1400 images belonging to 7 classes.


#### 

#### Predição e acurácia

In [16]:
new_modelo.fit(
    trainGen,
    steps_per_epoch=int(2284/32),#train_generator.samples // batch_size
    validation_data=testGen,
    epochs=10
)

Epoch 1/10
71/71 [==============================] - 34s 430ms/step - loss: 1.3439 - accuracy: 0.7238 - val_loss: 0.6942 - val_accuracy: 0.8514
Epoch 2/10
71/71 [==============================] - 29s 407ms/step - loss: 0.4671 - accuracy: 0.8939 - val_loss: 0.7266 - val_accuracy: 0.8486
Epoch 3/10
71/71 [==============================] - 29s 408ms/step - loss: 0.4493 - accuracy: 0.8988 - val_loss: 0.8869 - val_accuracy: 0.8521
Epoch 4/10
71/71 [==============================] - 29s 413ms/step - loss: 0.3756 - accuracy: 0.9192 - val_loss: 0.8714 - val_accuracy: 0.8450
Epoch 5/10
71/71 [==============================] - 31s 432ms/step - loss: 0.4442 - accuracy: 0.9050 - val_loss: 0.7460 - val_accuracy: 0.8793
Epoch 6/10
71/71 [==============================] - 28s 396ms/step - loss: 0.4201 - accuracy: 0.9121 - val_loss: 0.6443 - val_accuracy: 0.9050
Epoch 7/10
71/71 [==============================] - 31s 436ms/step - loss: 0.2911 - accuracy: 0.9405 - val_loss: 0.6840 - val_accuracy: 0.9079